# Geoviews Shape and Overlay 
Modified: Jun 6, 2019

- Goal: Incorporate vector tile services to geoviews project   

Overview:
1. geoview shape with shapely.geometry objects
2. create a hv.NdOverlay with a dictionary of gv.Shape objects
3. create a cmap cycler to color each shape by index
4. view it in a global basetile
5. add a latlon stream (from a basemap) to get a vectorile (from the tile service or cache) and plot the ndoverlay
object on top of the same basemap

In [92]:
%load_ext autoreload
%autoreload 2

import os, sys, time, math, json, requests
import numpy as np
import scipy as sp
import pandas as pd
import intake
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts


import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

In [ ]:
import math

class VectorTile():

    @staticmethod
    def deg2tile_xy(lat_deg, lon_deg, zoom):
        """
        Lat,Lon, z to tile numbers (xtile, ytile)
        - src: https://is.gd/mjvdR7
        """
        lat_rad = math.radians(lat_deg)
        n = 2.0 ** zoom
        xtile = int((lon_deg + 180.0) / 360.0 * n)
        ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
        return (xtile, ytile)

    @staticmethod
    def tile_xyz2deg(xtile, ytile, zoom):
        """
        Tile numbers to lat/lon in degree
        This returns the NW-corner of the square. 
        Use the function with xtile+1 and/or ytile+1 to get the other corners. 
        With xtile+0.5 & ytile+0.5 it will return the center of the tile.
        - src: https://is.gd/mjvdR7
        """
        n = 2.0 ** zoom
        lon_deg = xtile / n * 360.0 - 180.0
        lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
        lat_deg = math.degrees(lat_rad)
        return (lat_deg, lon_deg)

In [ ]:
import json

VECTILE_CACHE = {}
def get_vectile_gdf_at_xyz(xtile, ytile, z,
                    size=256,layer='all',
                    fformat='json', 
                    cache_dir='../data/vectile_cache/'):
    """
    Given xtile, ytile and z(oom level), 
    request the vector tile from nextzen vector tile endpoint
    
    If the tile was requested before and is saved, 
    it will check the current python session's cache, then the local
    disk to read the tile from memory.
    
    If not cached, it will send a request to the vector tile server,
    save the tile data both in python memory and local disk.
    
    Returns geopandas.DataFrame that contains some meta data like osm_id 
    and most importantly) geometries
    
    Args:
    - xtile, ytile, z (int)
    - size (int) : currently only supports 256 because of the latlon->tile
        conversion calculation is constrained to that size
    - fformat (str): currently it must be json because I don't know
        how to read mvt or topojson formats to geopandas.DataFrame
        
    """
    #check if VEC_CACHE object exists in global namespace
    global VECTILE_CACHE
    try:
        VECTILE_CACHE
    except NameError:
        VECTILE_CACHE = {}
        
    cache_key = (size,layer,z,xtile,ytile)
    # Check if this tile is in python session memory
    # If so, read from the memory, otherwise read from the disk
    if VECTILE_CACHE.get( cache_key ):
        if VECTILE_CACHE[cache_key].get('loaded'):
            "Reading from python session memory..."
            return VECTILE_CACHE[cache_key].get('dframe') #geopandas.gdf
        else:
            "Reading from disk cache..."
            return gpd.GeoDataFrame.read_file(VECTILE_CACHE[cache_key].get('fpath'))
    
    # Request a new tile
    print("Not in cache, sending request to vector tile service...")
    tile_url = f'https://tile.nextzen.org/tilezen/vector/v1/{size}/{layer}/{z}/{xtile}/{ytile}.{fformat}?api_key=GpjLSbvrQsa98MgMMuodzw'
    r = requests.get(url=tile_url)
    if not r.ok:
        raise ValueError('reponse not ok: ', r.status_code)
    data = r.json()
    
    # Write to disk
    fdir = (Path(cache_dir) / f'{size}/{layer}/{z}/{xtile}/').resolve()
    if not fdir.exists():
        fdir.mkdir(parents=True)
        print(f'{fdir} created')
    fpath = fdir/ f'{ytile}.{fformat}'
    print('Saving to: ', fpath)
    with open(fpath, 'w') as f:
        json.dump(data,f)
        
    while not fpath.exists():
        time.sleep(0.3)
    if fpath.exists():
        gdf = gpd.read_file(fpath)
    else:
        raise IOError('File was not correctly written to disk: ', fpath)
    
    # Write to cache
    VECTILE_CACHE[cache_key] = {
        'loaded': True,
        'dframe': gdf,
        'fpath': str(fpath)
    }
    return gdf


def get_vectile_overlay_at_xyz(xtile, ytile, z, **kwargs):
    """
    Fetches the vector tile (from python cache or from the local disk or from the web service <- search order)
    and returns a NdOverlay of Shape Elements with a numeric index
    
    args:
    - xtile, ytile, z (int)

        
    kwargs:    
    - colors (iterable): to be used to generate a itertools.cycle to cycle through 
        color values
        eg: color=bokeh.palettes.Category20_10
    - size: (default is 256) 
    - layer: (default is 'all')
    - fformat: (default is 'json')  
    - cache_dir: (default is '../data/vectile_cache/')
    """

    gdf = get_vectile_gdf_at_xyz(xtile, ytile, z, **kwargs)
    
    # colormap iterator
    import itertools
    from bokeh.palettes import Category20_10

    colors = kwargs.get('colors', Category20_10)
    cmap_cycler = itertools.cycle(colors)
    
    # return ndoverlay of each shape
    return hv.NdOverlay( {i:gv.Shape(geom).opts(fill_color=c) for i, (geom, c) in enumerate( zip(gdf.geometry, cmap_cycler) ) })

def get_vectile_overlay_at_latlon(lat, lon, z, **kwargs):
    """
    Args:
    - lat, lon (float): lat lon in degrees
    - z (int): zoom level
    """
    xtile, ytile = VectorTile.deg2tile_xy(lat, lon,z)
    return get_vectile_overlay_at_xyz(xtile, ytile, z, **kwargs)  

def relabel_overlay(ndOverlay, labels):
    """
    ndOverlay is indexed by integer
    length of hv elements in the overlay must equal to the length of labels
    """
    relabeled = hv.NdOverlay({i: ndOverlay[i].relabel(labels[i]) for i in range(len(ndOverlay))})
    return relabeled

## Tests
def test_get_vectile_gdf_at_xyz():
    x,y,z=(38229, 34597,16)
    gdf = get_vectile_gdf_at_xyz(x,y,z)
    vectile_overlay = hv.NdOverlay({i:gv.Shape(geom) for i,geom in enumerate(gdf.geometry)})
    display(vectile_overlay)
    

def test_get_vectile_overlay_at_xyz():
#     xtile=19293
#     ytile=24641
    
    xtile, ytile = (38420, 33268)
    z=10
    display(get_vectile_overlay_at_xyz(xtile, ytile,z))
    
    
def test_get_vectile_overlay_at_latlon():
#     lat, lon = (40.709792012434946, -74.0203857421875)
#     lat, lon = (-10, 30)
    lon, lat = 31.05, -2.75
    z=16
    display(get_vectile_overlay_at_latlon(lat, lon, z))
    
    
    

## Simple Maptile xyz viewer
https://jsfiddle.net/api/post/library/pure/

## Test 1


In [ ]:
z = 12
lon,lat = (22.35, -11.45)
xtile, ytile = VectorTile.deg2tile_xy(lat, lon, z)


print("xtile, ytile: ", xtile, ytile)

In [ ]:
temp = get_vectile_gdf_at_xyz(xtile, ytile, z)

In [ ]:
temp


## Test 2

In [ ]:
z, xtile, ytile = 14,8030,5424
temp = get_vectile_gdf_at_xyz(xtile, ytile, z)
temp

In [ ]:
temp.plot()

In [ ]:
lat, lon = VectorTile.tile_xyz2deg(xtile, ytile, z)
print(lat,lon)

get_vectile_overlay_at_xyz(xtile, ytile,z)

## Test 3: Khartoum

<img src="../assets/khartoum_maptile_info.png" alt="khartoum_maptile_info" width="1000"/>

1. get_vectile_gdf_at_xyz()

In [ ]:
k_lat, k_lon = 15.509138, 32.550624
z = 10
k_xtile, k_ytile = VectorTile.deg2tile_xy(k_lat, k_lon, z)
print(kx, ky)
k_gdf = get_vectile_gdf_at_xyz(k_xtile, k_ytile, z)
k_gdf.plot()

2. get_vectile_overlay_at_xyz

In [ ]:
k_overlay2 = get_vectile_overlay_at_xyz(k_xtile, k_ytile,z)

In [ ]:
k_overlay2

In [ ]:
print(k_overlay2)

In [ ]:
# We can index into the hv.Ndoverlay object with integer indexing (as to a dictionary)
# for i in range(len(k_overlay2[::2])):
#     s = k_overlay2[i].opts.hover(
#     display(s.name, s.label, s)

3. get_vectile_overlay_at_latlon

In [ ]:
k_overlay = get_vectile_overlay_at_latlon(k_lat, k_lon, z)
k_overlay

In [ ]:
geom = 

In [ ]:
s = k_overlay[9]
s.print_param_values()

In [ ]:
s.data


In [ ]:
s.data['geometry']

In [ ]:
# temp = gv.Shape(k_gdf.iloc[, kdims=['Longitude', 'Latitude'], vdims=
gv.Shape(k_gdf.iloc[[9]])

In [ ]:
k_gdf.columns

In [ ]:
s.label = '9'


In [ ]:
s.relabel('id9')

In [ ]:
relabel_overlay(k_overlay, k_gdf['name:en'])

In [ ]:
k_gdf['name:en']
